In [1]:
import tensorflow as tf
import numpy as np

random = np.random.random
randn = np.random.randn
Dataset = tf.data.Dataset

/home/bill/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# baisc

In [3]:
_seed = 1
np.random.seed(_seed)
array = np.random.randn(2,2)
array

array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862]])

#  
- convert_to_tensor
- from_tensors

In [4]:
_X = [tf.convert_to_tensor(randn(2,2)) for _ in range(10)]
X = Dataset.from_tensor_slices(tf.stack(_X))
_y = [tf.convert_to_tensor(random()) for _ in range(10)]
y = Dataset.from_tensor_slices(tf.stack(_y))

## zip

In [5]:
# zip
dataset = Dataset.zip((X,y))
dataset

<ZipDataset shapes: ((2, 2), ()), types: (tf.float64, tf.float32)>

In [6]:
dataset.__dict__

{'_datasets': (<TensorSliceDataset shapes: (2, 2), types: tf.float64>,
  <TensorSliceDataset shapes: (), types: tf.float32>)}

## batching

In [7]:
batched_dataset = dataset.batch(4)
batched_dataset

<BatchDataset shapes: ((?, 2, 2), (?,)), types: (tf.float64, tf.float32)>

batching 只是添加了另外一个dim维度

## repeat and shuffle

In [8]:
buffer_size = 2
n_dataset = dataset.repeat(2).shuffle(buffer_size)
n_dataset

<ShuffleDataset shapes: ((2, 2), ()), types: (tf.float64, tf.float32)>

In [9]:
dataset

<ZipDataset shapes: ((2, 2), ()), types: (tf.float64, tf.float32)>

---
# iterator

### one shot

In [10]:
_iter = n_dataset.make_one_shot_iterator()
a = _iter.get_next()
with tf.Session() as sess:
    N = 0
    while True:
        try:
#             print(N)
            sess.run(a)
            N += 1
        except tf.errors.OutOfRangeError:
            break

### initializable: dataset initialization  
dataset是同一个  
iterator must be initialized before using it.

In [11]:
_iter = n_dataset.make_initializable_iterator()
_iter.initializer
e = _iter.get_next()

In [12]:
with tf.Session() as sess:
    sess.run(_iter.initializer)
    while True:
        try:
            print(sess.run(e))
        except:
            break

(array([[ 0.3190391 , -0.24937038],
       [ 1.46210794, -2.06014071]]), 0.5358964)
(array([[-0.3224172 , -0.38405435],
       [ 1.13376944, -1.09989127]]), 0.66379464)
(array([[-0.17242821, -0.87785842],
       [ 0.04221375,  0.58281521]]), 0.5148891)
(array([[-1.10061918,  1.14472371],
       [ 0.90159072,  0.50249434]]), 0.94459474)
(array([[ 0.90085595, -0.68372786],
       [-0.12289023, -0.93576943]]), 0.58655506)
(array([[-0.26788808,  0.53035547],
       [-0.69166075, -0.39675353]]), 0.9034019)
(array([[-0.6871727 , -0.84520564],
       [-0.67124613, -0.0126646 ]]), 0.1374747)
(array([[ 0.86540763, -2.3015387 ],
       [ 1.74481176, -0.7612069 ]]), 0.04995346)
(array([[-1.11731035,  0.2344157 ],
       [ 1.65980218,  0.74204416]]), 0.13927634)
(array([[ 0.86540763, -2.3015387 ],
       [ 1.74481176, -0.7612069 ]]), 0.04995346)
(array([[ 0.3190391 , -0.24937038],
       [ 1.46210794, -2.06014071]]), 0.5358964)
(array([[-0.19183555, -0.88762896],
       [-0.74715829,  1.6924546 ]]

### reinitializable
iterator initialization
iterator 是同一个， dataset可以不同  
- Iterator.from_structure(output_types, output_shapes)
- make_initializer(dataset)


In [13]:
dataset1 = Dataset.range(2,8,3)
dataset2 = Dataset.range(1,5,2)
Iterator = tf.data.Iterator
_iter = Iterator.from_structure(dataset1.output_types, dataset1.output_shapes)
e = _iter.get_next()
init1 = _iter.make_initializer(dataset1)
init2 = _iter.make_initializer(dataset2)
sess = tf.Session()
sess.run(init1)
print(sess.run(e))
sess.run(init2)
print(sess.run(e))

2
1


### feedable
like generator: does not require you to initialize the iterator from the start of a dataset when you switch between iterators  
- resume state: using handle

In [14]:
iter1 = Dataset.range(2, 20, 3).repeat().make_one_shot_iterator()
iter2 = Dataset.range(23, 40, 3).make_initializable_iterator()

handle = tf.placeholder(tf.string, shape=[], name='handle')
_iter1 = Iterator.from_string_handle(handle, dataset1.output_types, dataset1.output_shapes)
e = _iter1.get_next()
sess = tf.Session()
h1, h2 = sess.run((iter1.string_handle(),iter2.string_handle()))

for _ in range(20):
    for _ in range(3):
        print(sess.run(e, feed_dict={handle: h1}))

    sess.run(iter2.initializer)
    for _ in range(3):
        print(sess.run(e, feed_dict={handle: h2}))

2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29
2
5
8
23
26
29
11
14
17
23
26
29


---
## ptb Dataset API
- convert_to_tensor  
- size
- Dataset.range(start, stop, step)
- set_shape(shape_tuple)
- dataset.batch(batch_size)
- make_one_shot_iterator()
- iterator.get_next()
- dataset.map(_func)

In [15]:
# ptb laguage data
batch_size = 2
num_steps = 2
raw_data = tf.convert_to_tensor(list(range(20)))
data_len = tf.size(raw_data)
raw_data = raw_data[0: data_len//num_steps * num_steps]
data_len = tf.size(raw_data)
_sample_begins = Dataset.range(0, tf.cast(data_len - 1, tf.int64), num_steps)

def _xy(i):
    "a element"
    x = raw_data[i: i+num_steps]
    x.set_shape((num_steps))
    y = raw_data[i+1: i+num_steps+1]
    y.set_shape((num_steps))
    return x, y
buffer_size = 100
with tf.Session().as_default():
    print(_sample_begins.repeat(2)
                        .shuffle(buffer_size)
                        .map(_xy)
                        .batch(batch_size)
                        .make_one_shot_iterator()
                        .get_next())

(<tf.Tensor 'IteratorGetNext_4:0' shape=(?, 2) dtype=int32>, <tf.Tensor 'IteratorGetNext_4:1' shape=(?, 2) dtype=int32>)
